In [1]:
import ee
import geemap
import geemap.colormaps as cm

import eemont
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [ ]:
point = ee.Geometry.Point(3.3896932773281856, 6.516974260768799) #Lagos, Nigeria
aoi = point.buffer(5000).bounds()


Map = geemap.Map()
Map.centerObject(aoi, 12)
Map.addLayer(aoi, {}, 'AOI')

No such comm: 3376f7b461c94c899405dd4686c7378f
No such comm: 3376f7b461c94c899405dd4686c7378f
No such comm: e198078c9f4e4bfa9198fb6bc878a0ea


In [5]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)

s2 = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterDate('2023-01-01', '2023-01-15')
    # Pre-filter to get less cloudy granules.
    #.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 60))
    .filterBounds(aoi)
    #.map(mask_s2_clouds)
    .maskClouds()
    .scaleAndOffset()
    .spectralIndices(['MNDVI', 'NDBI', 'MNDWI'])
    .mean()
    .clip(aoi)
)

vis_fcc = {
    'min': 0.2,
    'max': 0.5,
    'bands': ['B8', 'B4', 'B3'],
}


Map.add_layer(s2, vis_fcc, 'FCC')
# visualize spectral indices
Map.addLayer(s2.select('MNDVI'), {'min': -0.2, 'max': 0.3, 'palette': cm.palettes.ndvi}, 'MNDVI')
Map.addLayer(s2.select('NDBI'), {'min': -0.2, 'max': 0.1, 'palette': cm.get_palette('coolwarm')}, 'NDBI')
Map.addLayer(s2.select('MNDWI'), {'min': -0.3, 'max': 0.1, 'palette': cm.get_palette('RdBu')}, 'MNDWI')
Map

Map(center=[6.5169822257490875, 3.389765029252635], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
Map.user_rois.getInfo()

### ESRI LULC Link: https://gee-community-catalog.org/projects/S2TSLULC/

In [7]:
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B11', 'B12', 'MNDVI', 'NDBI', 'MNDWI']
image = s2.select(bands)

# create a reference image and sample from it
esri_lulc = (ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS")
                .filterDate('2022-01-01', '2022-12-31')
                .mosaic()
                .clip(aoi)
                .remap([1,2,4,5,7,8,9,10,11],[1,2,3,4,5,6,7,8,9])
)

esri_vis = {
  "names": [
    "Water",
    "Trees",
    "Flooded Vegetation",
    "Crops",
    "Built Area",
    "Bare Ground",
    "Snow/Ice",
    "Clouds",
    "Rangeland"
  ],
  "colors": [
    "#1A5BAB",
    "#358221",
    "#87D19E",
    "#FFDB5C",
    "#ED022A",
    "#EDE9E4",
    "#F2FAFF",
    "#C8C8C8",
    "#C6AD8D"
  ]}
Map.addLayer(esri_lulc, {'min':1, 'max':9, 'palette':esri_vis['colors']}, '2022 LULC 10m', False)

new_esri = esri_lulc.remap([1, 5],[2, 1]).rename('lc')

Map.addLayer(new_esri, {'min':1, 'max':2, 'palette':['red', 'blue']}, 'NEW ESRI', False)
Map


Map(bottom=1010801.0, center=[6.5218772856587, 3.382150599504441], controls=(WidgetControl(options=['position'…

In [9]:
# projections of image and new reference image
print('Image projection:', image.projection().getInfo())
print('Reference projection:', new_esri.projection().getInfo())

Image projection: {'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}
Reference projection: {'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}


In [8]:
# generate random samples (stratifed by class)
sample_size = 1000
seed = 42
scale = 10
samples = new_esri.stratifiedSample(
    numPoints=sample_size,
    classBand='lc',
    region=aoi,
    scale=scale,
    projection=image.projection(),
    seed=seed,
    geometries=True
)

Map.addLayer(samples, {}, 'samples', False)
print(samples.size().getInfo())
print(samples.first().getInfo())
Map

2000
{'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Point', 'coordinates': [3.3707035248374746, 6.478604913305783]}, 'id': '0', 'properties': {'lc': 1}}


Map(bottom=1010801.0, center=[6.5218772856587, 3.382150599504441], controls=(WidgetControl(options=['position'…

In [10]:
# sample the image
samples = image.sampleRegions(
    collection=samples,
    properties=['lc'],
    scale=scale,
    projection=image.projection(),
    geometries=True
)
samples.first().getInfo()

{'type': 'Feature',
 'geometry': {'geodesic': False,
  'type': 'Point',
  'coordinates': [3.3707035248374746, 6.478604913305783]},
 'id': '0_0',
 'properties': {'B11': 0.28350000000000003,
  'B12': 0.2403,
  'B2': 0.201,
  'B3': 0.22660000000000002,
  'B4': 0.23570000000000002,
  'B5': 0.26195,
  'B6': 0.28790000000000004,
  'B7': 0.30425,
  'B8': 0.2982,
  'MNDVI': 0.10819010776457585,
  'MNDWI': -0.11102368908281263,
  'NDBI': -0.025656033147810106,
  'lc': 1}}

In [11]:
# split the samples into training and testing
split = 0.7
samples = samples.randomColumn()

training = samples.filter(ee.Filter.lt('random', split))
testing = samples.filter(ee.Filter.gte('random', split))


# train a classifier
classifier = ee.Classifier.smileRandomForest(numberOfTrees=100).train(
    features=training,
    classProperty='lc',
    inputProperties=bands
)

# classify the image
classified = image.select(bands).classify(classifier)
Map.addLayer(classified, {'min':1, 'max':2, 'palette':['red', 'blue']}, 'classified')
Map

Map(bottom=2021323.0, center=[6.52008650241497, 3.368124961853028], controls=(WidgetControl(options=['position…

In [12]:
# Train Accuracy

train_accuracy = classifier.confusionMatrix()
print('Train Confusion Matrix:', train_accuracy.getInfo())

print('Train Overall Accuracy:', train_accuracy.accuracy().getInfo())

print('Producer\'s Accuracy:', train_accuracy.producersAccuracy().getInfo())
print('Consumer\'s Accuracy:', train_accuracy.consumersAccuracy().getInfo())



# Test Accuracy
test_accuracy = testing.classify(classifier).errorMatrix('lc', 'classification')

print('Test Confusion Matrix:', test_accuracy.getInfo())
print('Test Overall Accuracy:', test_accuracy.accuracy().getInfo())
print('Producer\'s Accuracy:', test_accuracy.producersAccuracy().getInfo())
print('Consumer\'s Accuracy:', test_accuracy.consumersAccuracy().getInfo())


Train Confusion Matrix: [[0, 0, 0], [0, 718, 0], [0, 0, 695]]
Train Overall Accuracy: 1
Producer's Accuracy: [[0], [1], [1]]
Consumer's Accuracy: [[0, 1, 1]]
Test Confusion Matrix: [[0, 0, 0], [0, 280, 0], [0, 3, 302]]
Test Overall Accuracy: 0.9948717948717949
Producer's Accuracy: [[0], [1], [0.9901639344262295]]
Consumer's Accuracy: [[0, 0.9893992932862191, 1]]
